<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

In [2]:
# Parameters
input_geojson_path = "training-data/postcards/mana-north_postcard.geojson"
output_csv_path = "automated_notebook_runs/sampled_data_csvs/mana-north_postcard_sampled_data.csv"


## Step 1.1: Configure the environment

In [3]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [4]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [5]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "suva_west"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-suva_west-27072025


In [6]:
gdfs = []
postcards_path = "training-data/postcards"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [7]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-efate-27072025_postcard.geojson
suva_west_postcard.geojson
paunganisu_postcard.geojson
mana-north_postcard.geojson
tulagi_postcard.geojson
serua_2_postcard.geojson
suva_central_postcard.geojson
efate_postcard.geojson
suva_east_postcard.geojson
bootless-bay_postcard.geojson
serua_postcard.geojson
korotogo_postcard.geojson
yanuca_postcard.geojson
mana-south_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [8]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [9]:
filename = "nm-efate-27072025_postcard.geojson"

In [10]:
training = gpd.read_file(f"training-data/postcards/{site}_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [11]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 36MB
Dimensions:      (y: 1101, x: 904, time: 1)
Coordinates:
  * y            (y) float64 9kB -2.036e+06 -2.036e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 7kB 3.154e+06 3.154e+06 ... 3.163e+06 3.163e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 2MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    red          (time, y, x) uint16 2MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    blue         (time, y, x) uint16 2MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    green        (time, y, x) uint16 2MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    emad         (time, y, x) float32 4MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    smad         (time, y, x) float32 4MB dask.array<chunksize=(1, 1101, 904), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 2MB dask.array<chu

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37993 instead
  warnings.warn(


In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled

<xarray.Dataset> Size: 108MB
Dimensions:      (y: 1101, x: 904)
Coordinates:
  * y            (y) float64 9kB -2.036e+06 -2.036e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 7kB 3.154e+06 3.154e+06 ... 3.163e+06 3.163e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 8MB 0.3823 0.3549 0.3757 ... 0.0254 0.0254
    red          (y, x) float64 8MB 0.0382 0.0385 0.0391 ... 0.0278 0.0278
    blue         (y, x) float64 8MB 0.0431 0.044 0.0452 ... 0.0526 0.0525 0.0525
    green        (y, x) float64 8MB 0.0647 0.0627 0.068 ... 0.0376 0.0378 0.0378
    emad         (y, x) float32 4MB 0.1156 0.1217 0.1082 ... 0.07745 0.07745
    smad         (y, x) float32 4MB 2.064e-07 2.648e-07 ... 5.003e-06 5.003e-06
    ...           ...
    swir16       (y, x) float64 8MB 0.1616 0.1633 0.164 ... 0.0224 0.0223 0.0223
    swir22       (y, x) float64 8MB 0.076 0.0768 0.0775 ... 0.0203 0.0203 0.0203
    coastal      (y, x) float64 8MB 0.0436 0.0447 0.045 ... 0.0611 0.0611 0.0611
    rededge1     (y, x) float64 8MB 0.0941 0.097 0.0972 ... 0.0287 0.0286 0.0286
    rededge2     (y, x) float64 8MB 0.2814 0.2882 0.2886 ... 0.0264 0.0264
    rededge3     (y, x) float64 8MB 0.3483 0.3508 0.3512 ... 0.0264 0.0264

In [17]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [18]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [19]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [20]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 231MB
Dimensions:      (y: 1101, x: 904)
Coordinates:
  * y            (y) float64 9kB -2.036e+06 -2.036e+06 ... -2.047e+06 -2.047e+06
  * x            (x) float64 7kB 3.154e+06 3.154e+06 ... 3.163e+06 3.163e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 8MB 0.3823 0.3549 0.3757 ... 0.0254 0.0254
    red          (y, x) float64 8MB 0.0382 0.0385 0.0391 ... 0.0278 0.0278
    blue         (y, x) float64 8MB 0.0431 0.044 0.0452 ... 0.0526 0.0525 0.0525
    green        (y, x) float64 8MB 0.0647 0.0627 0.068 ... 0.0376 0.0378 0.0378
    emad         (y, x) float32 4MB 0.1156 0.1217 0.1082 ... 0.07745 0.07745
    smad         (y, x) float32 4MB 2.064e-07 2.648e-07 ... 5.003e-06 5.003e-06
    ...           ...
    homogeneity  (y, x) float32 4MB 0.9583 0.9583 0.9583 0.9583 ... nan nan nan
    energy       (y, x) float32 4MB 0.9048 0.878 0.8521 0.8396 ... nan nan nan
    ASM          (y, x) float32 4MB 0.8187 0.7708 0.7261 0.7049 ... nan nan nan
    correlation  (y, x) float32 4MB 0.2059 0.4545 0.5781 0.619 ... nan nan nan
    mean         (y, x) float32 4MB 1.056 1.083 1.111 1.125 ... nan nan nan nan
    entropy      (y, x) float32 4MB 0.601 0.7417 0.8519 0.9 ... nan nan nan nan

In [21]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [22]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [23]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2045225.0,3163165.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2044855.0,3162955.0,3832,2024-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2045365.0,3162785.0,3832,2024-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2044885.0,3162385.0,3832,2024-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2044385.0,3162605.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2046515.0,3162245.0,3832,2024-01-01
298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2046225.0,3161695.0,3832,2024-01-01
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2046095.0,3161505.0,3832,2024-01-01


In [24]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
41,9,0.0355,0.0583,0.0791,0.0889,0.116923,4.171675e-06,0.000025,0.0341,0.0653,...,1.000000,1.000000,1.000000,1.000000,3.000000,-1.442695e-10,-2043995.0,3162965.0,3832,2024-01-01
42,9,0.0304,0.0558,0.0771,0.0893,0.089727,5.277067e-06,0.000027,0.0285,0.0503,...,1.000000,1.000000,1.000000,1.000000,3.000000,-1.442695e-10,-2043895.0,3162805.0,3832,2024-01-01
95,5,0.0399,0.0468,0.0533,0.0579,0.074773,1.578349e-06,0.000019,0.0481,0.1889,...,0.465708,0.249227,0.062114,0.736506,7.236111,5.173962e+00,-2037705.0,3162815.0,3832,2024-01-01
96,5,0.0352,0.0477,0.0515,0.0567,0.075187,9.162187e-07,0.000016,0.0594,0.1171,...,0.576992,0.234266,0.054880,0.749694,5.777778,5.087485e+00,-2037725.0,3162825.0,3832,2024-01-01
97,5,0.0918,0.0799,0.0715,0.0825,0.129386,1.963276e-06,0.000019,0.0959,0.1479,...,0.916667,0.616923,0.380594,0.689432,1.652778,1.741967e+00,-2037675.0,3162745.0,3832,2024-01-01


In [25]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [26]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [27]:
training_array=training_array[standard_schema]

In [28]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [29]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [30]:
training_array["cc_id"].dtype

dtype('int32')